<a href="https://colab.research.google.com/github/GNunes-br/nlp_sentiment_analysis_pt3/blob/main/Machine_Learning_para_classificar_textos_Parte_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 6.3 MB/s eta 0:00:00


In [28]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import nltk
from nltk import tokenize

import seaborn as sns

from string import punctuation

import unidecode

In [2]:
driveSource = '/content/drive/MyDrive/Colab Notebooks/datasets'

In [4]:
dataset = pd.read_csv(driveSource + "/imdb_pt_br.csv")

In [5]:
dataset

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg
...,...,...,...,...
49454,49456,"Seeing as the vote average was pretty low, and...","Como a média de votos era muito baixa, e o fat...",pos
49455,49457,"The plot had some wretched, unbelievable twist...",O enredo teve algumas reviravoltas infelizes e...,pos
49456,49458,I am amazed at how this movieand most others h...,Estou espantado com a forma como este filme e ...,pos
49457,49459,A Christmas Together actually came before my t...,A Christmas Together realmente veio antes do m...,pos


In [21]:
classification = dataset.sentiment.replace(['neg', 'pos'], [0,1])
dataset["classif"] = classification

In [61]:
nltk.download('all', quiet=True)

True

In [62]:
def getStopWordsList():

  return nltk.corpus.stopwords.words('portuguese')  

In [63]:
def getPunctuationList():
  punctuationList = list()

  for value in punctuation:
    punctuationList.append(value)

  return punctuationList

In [82]:
stopWords = [unidecode.unidecode(text) for text in (getStopWordsList() + getPunctuationList())]

In [83]:
wordPunctTokenizer = tokenize.WordPunctTokenizer()

In [84]:
whitespaceTokenizer = tokenize.WhitespaceTokenizer()

In [87]:
stemmer = nltk.RSLPStemmer()

In [91]:
processedText = list();

for text in dataset.text_pt:
  newPhrase = list()

  convertTextToLowercase = text.lower()

  removeTextAccents = unidecode.unidecode(convertTextToLowercase)

  textWordPunctTokenizer = wordPunctTokenizer.tokenize(removeTextAccents)

  textWhiteSpaceTokenizer = whitespaceTokenizer.tokenize(' '.join(textWordPunctTokenizer))

  textWords = textWhiteSpaceTokenizer

  for word in textWords:
    if word not in stopWords:
      newPhrase.append(stemmer.stem(word))

  processedText.append(' '.join(newPhrase))

In [92]:
dataset['processed'] = processedText

In [93]:
tfidf = TfidfVectorizer(lowercase=False, ngram_range=(1,2))

In [139]:
tfidfVector = tfidf.fit_transform(dataset.processed)

train, test, trainClass, testClass = train_test_split(tfidfVector, dataset["classif"], random_state=42)

In [95]:
logisticRegression = LogisticRegression()

In [96]:
logisticRegression.fit(train, trainClass)

LogisticRegression()

In [142]:
acuracy = logisticRegression.score(test, testClass)

In [143]:
acuracy

0.8856449656287909

In [157]:
testClass[0:10]

12532    1
35445    0
20279    1
2969     0
45161    1
42648    1
17897    1
41126    1
47609    1
44181    1
Name: classif, dtype: int64

In [156]:
logisticRegression.predict(test[0:10])

array([1, 0, 0, 0, 1, 1, 1, 1, 1, 1])

In [163]:
dataset.query('id == 2969')

,id,text_en,text_pt,sentiment,classif,processed
2967,2969,This movie was one of the most boring horror m...,Este filme foi um dos filmes de terror mais en...,neg,0,film film terr entedi vi temp vist pessoal gos...


In [146]:
# logisticRegression.predict(tfidfVector)

logisticRegression.score(test, testClass)

(2, 12)

In [106]:
testClass

12532    1
35445    0
20279    1
2969     0
45161    1
        ..
16421    1
39861    1
309      0
20638    1
38935    1
Name: classif, Length: 12365, dtype: int64